In [33]:
import opentrons.execute
from opentrons import protocol_api

metadata = {'apiLevel': '2.13'}

In [40]:
from opentrons import simulate
# protocol = simulate.get_protocol_api(metadata['apiLevel']) # uncomment to run in simulation

protocol = opentrons.execute.get_protocol_api(metadata['apiLevel']) # uncomment to run on the actual hardware


In [41]:
# Load labware
plate = protocol.load_labware('corning_96_wellplate_360ul_flat', 1)
block = protocol.load_labware('opentrons_24_aluminumblock_nest_1.5ml_snapcap', 2)
tiprack = protocol.load_labware('opentrons_96_tiprack_300ul', 8)
pipette = protocol.load_instrument('p300_single', 'right',
                                     tip_racks=[tiprack])


NameError: name 'tip_rack_1' is not defined

In [42]:
pipette.starting_tip = tiprack.well('D1')

In [45]:
def serial_dilution(dye, water, destination_wells, dilution_factor=0.5, final_volume=50):
    assert 0 < dilution_factor < 1
    transfer_volume = final_volume * dilution_factor / (1-dilution_factor)
    transient_volume = final_volume + transfer_volume
    vol_water = final_volume
    vol_dye = transfer_volume
    
    print('Transient volume: ', transient_volume, '; Volume water: ', vol_water, '; Volume Dye: ', vol_dye)
    
    pipette.pick_up_tip() 
    
    # Per-fill with water
    pipette.distribute(vol_water, water, destination_wells, new_tip='never')
    
    # For each destination well, calcualte where we will take liquid from 
    source_wells = [dye] + destination_wells[:-1]
    print('Source wells:', source_wells)
    
    # Pre-mix dye container
    pipette.mix(3, 150, dye)
    for source_well, dest_well in zip(source_wells, destination_wells):
        pipette.aspirate(vol_dye, source_well)
        pipette.dispense(vol_dye, dest_well)
        pipette.mix(3, transient_volume*0.8, dest_well)
    
    pipette.drop_tip()

In [46]:
dest_wells = plate.rows_by_name()['E'][:5]
print('Destination wells:', dest_wells)
serial_dilution(dye=block['A1'], water=block['A2'], destination_wells=dest_wells, final_volume=50, dilution_factor=0.7)

Destination wells: [E1 of Corning 96 Well Plate 360 µL Flat on 1, E2 of Corning 96 Well Plate 360 µL Flat on 1, E3 of Corning 96 Well Plate 360 µL Flat on 1, E4 of Corning 96 Well Plate 360 µL Flat on 1, E5 of Corning 96 Well Plate 360 µL Flat on 1]
Transient volume:  166.66666666666663 ; Volume water:  50 ; Volume Dye:  116.66666666666664
Source wells: [A1 of Opentrons 24 Well Aluminum Block with NEST 1.5 mL Snapcap on 2, E1 of Corning 96 Well Plate 360 µL Flat on 1, E2 of Corning 96 Well Plate 360 µL Flat on 1, E3 of Corning 96 Well Plate 360 µL Flat on 1, E4 of Corning 96 Well Plate 360 µL Flat on 1]


<InstrumentContext: p300_single_v2.1 in RIGHT>